In [ ]:
from datetime import datetime
from datetime import timezone

class Checkin:
    def __init__(self, id, createdAt, tzOffSet, venue):
        self.id = id
        self.utcDate = datetime.fromtimestamp(createdAt + (60 * tzOffSet), timezone.utc)
        self.venue = venue
        
    def __iter__(self):
        return iter([self.id, self.utcDate, self.venue.id, self.venue.name, self.venue.lat, self.venue.long, self.venue.city, self.venue.country])

class Venue:
    def __init__(self, id):
        self.id = id
        self.name = ''
        self.lat = ''
        self.long = ''
        self.city = ''
        self.country = '' 
    
    def addDetails(self, name, lat, long, city, country):
        self.name = name
        self.lat = lat
        self.long = long
        self.city = city
        self.country = country       

In [ ]:
# load json checkins data from 4SQ export
import json

checkins = {}
venues = {}
   
print("load checkins data from 4SQ export file")

with open("checkins.json", "r") as read_file:
    fQcheckins = json.load(read_file)
    
    # iterate on checkins data
    for fQCheckin in fQcheckins["items"]:
        checkinId = fQCheckin["id"]
        #str(print(checkinId)
        
        # save id venue for future processing
        venue4q = fQCheckin.get('venue')
        if venue4q is not None:
            venueId = venue4q['id']
            if venues.get(venueId) is None:
                venue = Venue(venueId)
                venues[venueId] = venue
            else:
                venue = venues.get(venueId)
        
            # save checkins data
            checkin = Checkin(checkinId, fQCheckin["createdAt"], fQCheckin["timeZoneOffset"], venue)
            checkins[checkinId] = checkin
        
print("numbers of venues :", len(venues))
print("numers of checkins:", len(checkins))       

In [ ]:
import requests

headers = {"Accept": "application/json", "Accept-Language": "en" }
url = "https://api.foursquare.com/v2/venues/{0}/?v=20220829&client_id={yourclientid}&client_secret={client_secret}"

count=0
# now we can process the list of venues to get details from Foursquare API
for venueId, venue in venues.items():
    urlWithVenueId = url.format(venueId)
    response = requests.get(urlWithVenueId, headers=headers)
    
    count+=1
    print(count)
    
    if response.ok:
        venueData = json.loads(response.text)["response"]["venue"]
        venueLocation = venueData["location"]
        # print(venueLocation)
        venue.addDetails(venueData["name"], venueLocation.get("lat"), venueLocation.get("lng"), venueLocation.get("city"), venueLocation.get("country"))
    else:
        print(response.status_code)
        print(response.text)
        
print("Venues processed")

In [ ]:
import csv

# save the data in a file now
with open('data-english-names.csv', 'w') as csvfile:
    csv_writer = csv.writer(csvfile)
    count = 0
    for checkinId, checkin in checkins.items():
        count+=1
        csv_writer.writerow(list(checkin)) 

print("lines saved:", count)
print("File saved")

In [ ]:
import requests
import json

headers = {"Accept": "application/json", "Accept-Language": "en" }
url = "https://api.foursquare.com/v2/venues/{0}/?v=20220829&client_id={client_id}&client_secret={client_secret}"

urlWithVenueId = url.format("518c4b7f5019d968d0177afe")
response = requests.get(urlWithVenueId, headers=headers)
    
if response.ok:
    venueData = json.loads(response.text)["response"]["venue"]
    venueLocation = venueData["location"]
    print(venueLocation) 
else:
    print(response.status_code)
    print(response.text)
        
print("Venues processed")